In [5]:
pip install yfinance pandas


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 77.5 MB/s  0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15636 sha256=7454a641fc12c135389e8c3ca7060952af2dc642bd1e7476fe226a334cf92559
  Stored in directory: /home/codespace/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
Successfully built multitasking
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/7 [websockets]  WARNING: The script websockets is installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━ 6/7 [yfinance]]  WARNING: The script samp

In [ ]:
assets = ['NIFTYBEES.NS','SILVERIETF.NS','GOLDBEES.NS','BANKBEES.NS']

In [7]:
import yfinance as yf
import pandas as pd

assets = ['NIFTYBEES.NS', 'SILVERIETF.NS', 'GOLDBEES.NS', 'BANKBEES.NS']

# Download last 5 years of weekly data
df = yf.download(
    tickers=assets,
    period="5y",
    interval="1wk",
    group_by="ticker",
    auto_adjust=False,
    progress=False
)

# Convert to a clean long-format DataFrame
weekly_data = []

for ticker in assets:
    temp = df[ticker].copy()
    temp['Ticker'] = ticker
    weekly_data.append(temp)

weekly_df = pd.concat(weekly_data)
weekly_df.reset_index(inplace=True)

weekly_df.head()


Price,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,2021-01-18,155.419998,169.699997,153.699997,153.910004,153.910004,5554785.0,NIFTYBEES.NS
1,2021-01-25,174.880005,174.880005,140.199997,146.100006,146.100006,12720778.0,NIFTYBEES.NS
2,2021-02-01,150.000000,174.699997,146.240005,159.050003,159.050003,19098337.0,NIFTYBEES.NS
3,2021-02-08,179.699997,179.699997,159.350006,162.169998,162.169998,10289663.0,NIFTYBEES.NS
4,2021-02-15,163.169998,172.589996,159.550003,160.419998,160.419998,9223996.0,NIFTYBEES.NS


In [8]:

# 1) Extract Adjusted Close prices into a wide table (index=Date, columns=Ticker)
prices = pd.DataFrame({t: df[t]["Adj Close"] for t in assets})

# 2) Compute weekly simple returns
rets = prices.pct_change().dropna()

# (Optional) If you prefer log returns:
# rets = np.log(prices / prices.shift(1)).dropna()

# 3) Covariance matrix (weekly)
cov_weekly = rets.cov()

# 4) Correlation matrix (often useful too)
corr = rets.corr()

# 5) Annualize covariance (weekly → annual), if needed
cov_annual = cov_weekly * 52

cov_weekly, cov_annual, corr

(               NIFTYBEES.NS  SILVERIETF.NS  GOLDBEES.NS  BANKBEES.NS
 NIFTYBEES.NS       0.000273       0.000031    -0.000007     0.000296
 SILVERIETF.NS      0.000031       0.001444     0.000544     0.000047
 GOLDBEES.NS       -0.000007       0.000544     0.000376    -0.000007
 BANKBEES.NS        0.000296       0.000047    -0.000007     0.000438,
                NIFTYBEES.NS  SILVERIETF.NS  GOLDBEES.NS  BANKBEES.NS
 NIFTYBEES.NS       0.014175       0.001613    -0.000389     0.015386
 SILVERIETF.NS      0.001613       0.075072     0.028279     0.002463
 GOLDBEES.NS       -0.000389       0.028279     0.019547    -0.000344
 BANKBEES.NS        0.015386       0.002463    -0.000344     0.022787,
                NIFTYBEES.NS  SILVERIETF.NS  GOLDBEES.NS  BANKBEES.NS
 NIFTYBEES.NS       1.000000       0.049435    -0.023348     0.856088
 SILVERIETF.NS      0.049435       1.000000     0.738223     0.059547
 GOLDBEES.NS       -0.023348       0.738223     1.000000    -0.016307
 BANKBEES.NS      

In [11]:
'''
import json
from datetime import datetime

assets = ["NIFTYBEES.NS", "SILVERIETF.NS", "GOLDBEES.NS", "BANKBEES.NS"]

invested_amounts = {
    "NIFTYBEES.NS": 254224,
    "SILVERIETF.NS": 63175,
    "GOLDBEES.NS": 91636,
    "BANKBEES.NS": 75419
}

payload = {
    "as_of": datetime.now().strftime("%Y-%m-%d"),
    "assets": assets,
    "invested_amounts": invested_amounts,
    "total_invested": sum(invested_amounts.values()),
    "data_settings": {
        "period": "5y",
        "interval": "1wk",
        "price_field": "Adj Close",
        "annualization_factor": 52
    }
}

# Write to file
with open("portfolio.json", "w", encoding="utf-8") as f:
    json.dump(payload, f, indent=2)

print("Saved portfolio.json")
print(json.dumps(payload, indent=2))
'''


'\nimport json\nfrom datetime import datetime\n\nassets = ["NIFTYBEES.NS", "SILVERIETF.NS", "GOLDBEES.NS", "BANKBEES.NS"]\n\ninvested_amounts = {\n    "NIFTYBEES.NS": 254224,\n    "SILVERIETF.NS": 63175,\n    "GOLDBEES.NS": 91636,\n    "BANKBEES.NS": 75419\n}\n\npayload = {\n    "as_of": datetime.now().strftime("%Y-%m-%d"),\n    "assets": assets,\n    "invested_amounts": invested_amounts,\n    "total_invested": sum(invested_amounts.values()),\n    "data_settings": {\n        "period": "5y",\n        "interval": "1wk",\n        "price_field": "Adj Close",\n        "annualization_factor": 52\n    }\n}\n\n# Write to file\nwith open("portfolio.json", "w", encoding="utf-8") as f:\n    json.dump(payload, f, indent=2)\n\nprint("Saved portfolio.json")\nprint(json.dumps(payload, indent=2))\n'

In [12]:
pip install streamlit yfinance pandas numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 76.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.4/795.4 kB 47.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 78.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 86.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 84.9 MB/s  0:00:006m0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━ 11/12 [streamlit]  WARNING: The script streamlit is installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12/12 [streamlit]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import json
import numpy as np
import pandas as pd
import yfinance as yf
import streamlit as st
from datetime import datetime

JSON_PATH = "portfolio.json"

def load_portfolio_json(path: str) -> dict | None:
    if not os.path.exists(path):
        return None
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def save_portfolio_json(path: str, payload: dict) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(payload, f, indent=2)

def compute_portfolio_variance(assets, invested_amounts, period="5y", interval="1wk",
                               price_field="Adj Close", annualization_factor=52):
    # Download data
    df = yf.download(
        tickers=assets,
        period=period,
        interval=interval,
        group_by="ticker",
        auto_adjust=False,
        progress=False
    )

    # Build price table: Date x Ticker
    prices = pd.DataFrame({t: df[t][price_field] for t in assets}).sort_index()

    # Returns
    rets = prices.pct_change().dropna()

    # Covariance matrices
    cov_weekly = rets.cov()
    cov_annual = cov_weekly * annualization_factor

    # Weights
    amounts = pd.Series(invested_amounts, dtype="float64")
    w = (amounts / amounts.sum()).reindex(assets)

    # Align Sigma with assets
    Sigma_w = cov_weekly.reindex(index=assets, columns=assets)
    Sigma_a = cov_annual.reindex(index=assets, columns=assets)

    # Portfolio variance and volatility
    var_weekly = float(w.T @ Sigma_w @ w)
    var_annual = float(w.T @ Sigma_a @ w)

    vol_weekly = float(np.sqrt(var_weekly))
    vol_annual = float(np.sqrt(var_annual))

    return {
        "prices": prices,
        "returns": rets,
        "weights": w,
        "cov_weekly": cov_weekly,
        "cov_annual": cov_annual,
        "var_weekly": var_weekly,
        "vol_weekly": vol_weekly,
        "var_annual": var_annual,
        "vol_annual": vol_annual
    }

st.set_page_config(page_title="Portfolio Variance Calculator", layout="wide")
st.title("Portfolio Variance Calculator (Covariance Matrix Method)")

portfolio = load_portfolio_json(JSON_PATH)

st.sidebar.header("Portfolio Source")
use_json = portfolio is not None

if use_json:
    st.sidebar.success(f"Found {JSON_PATH} — using it.")
    if st.sidebar.button("Ignore JSON and enter manually"):
        portfolio = None
        use_json = False
else:
    st.sidebar.warning(f"No {JSON_PATH} found — please enter manually.")

st.sidebar.header("Data Settings")
period = st.sidebar.text_input("period", value=(portfolio.get("data_settings", {}).get("period", "5y") if portfolio else "5y"))
interval = st.sidebar.text_input("interval", value=(portfolio.get("data_settings", {}).get("interval", "1wk") if portfolio else "1wk"))
price_field = st.sidebar.selectbox("price_field", ["Adj Close", "Close", "Open", "High", "Low"], index=0)
annualization_factor = st.sidebar.number_input("annualization_factor", value=int(portfolio.get("data_settings", {}).get("annualization_factor", 52) if portfolio else 52), min_value=1)

if not use_json:
    st.subheader("Manual Input")
    st.write("Enter tickers + invested amounts. Add/remove rows as needed.")

    default_rows = [
        {"Ticker": "NIFTYBEES.NS", "Invested": 254224},
        {"Ticker": "SILVERIETF.NS", "Invested": 63175},
        {"Ticker": "GOLDBEES.NS", "Invested": 91636},
        {"Ticker": "BANKBEES.NS", "Invested": 75419},
    ]

    edited = st.data_editor(
        pd.DataFrame(default_rows),
        num_rows="dynamic",
        use_container_width=True
    )

    # Clean input
    edited = edited.dropna()
    edited["Ticker"] = edited["Ticker"].astype(str).str.strip()
    edited = edited[edited["Ticker"] != ""]
    edited["Invested"] = pd.to_numeric(edited["Invested"], errors="coerce")
    edited = edited.dropna(subset=["Invested"])

    assets = edited["Ticker"].tolist()
    invested_amounts = dict(zip(edited["Ticker"], edited["Invested"]))

    if st.button("Save as portfolio.json"):
        payload = {
            "as_of": datetime.now().strftime("%Y-%m-%d"),
            "assets": assets,
            "invested_amounts": invested_amounts,
            "total_invested": float(sum(invested_amounts.values())),
            "data_settings": {
                "period": period,
                "interval": interval,
                "price_field": price_field,
                "annualization_factor": int(annualization_factor)
            }
        }
        save_portfolio_json(JSON_PATH, payload)
        st.success(f"Saved {JSON_PATH}. Refresh page to use it automatically.")
else:
    assets = portfolio["assets"]
    invested_amounts = portfolio["invested_amounts"]

st.subheader("Run Calculation")
if st.button("Compute portfolio variance"):
    try:
        res = compute_portfolio_variance(
            assets=assets,
            invested_amounts=invested_amounts,
            period=period,
            interval=interval,
            price_field=price_field,
            annualization_factor=annualization_factor
        )

        col1, col2 = st.columns(2)

        with col1:
            st.markdown("### Weights")
            st.dataframe((res["weights"] * 100).round(4).to_frame("Weight (%)"), use_container_width=True)

            st.markdown("### Variance / Volatility")
            st.write(f"**Weekly variance:** {res['var_weekly']:.8f}")
            st.write(f"**Weekly volatility:** {res['vol_weekly']:.4%}")
            st.write(f"**Annual variance:** {res['var_annual']:.8f}")
            st.write(f"**Annual volatility:** {res['vol_annual']:.4%}")

        with col2:
            st.markdown("### Covariance Matrix (Weekly)")
            st.dataframe(res["cov_weekly"].round(8), use_container_width=True)

            st.markdown("### Correlation Matrix (Weekly)")
            st.dataframe(res["returns"].corr().round(4), use_container_width=True)

        st.markdown("### Returns sample")
        st.dataframe(res["returns"].tail(10), use_container_width=True)

    except Exception as e:
        st.error(f"Error: {e}")


In [ ]:
python -m pip install --upgrade pip
pip install -r requirements.txt
streamlit run app.py --server.address 0.0.0.0 --server.port 8501


cd portfolio-management
ls -la
python -m pip install --upgrade pip
pip install -r requirements.txt
streamlit run app.py --server.address 0.0.0.0 --server.port 8501
